### Just 2 classes at the moment

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from sklearn.metrics import accuracy_score
from collections import defaultdict
from scipy.stats import norm
import operator
import time

In [2]:
# Generate Data
mean1 = [0]
cov1 = [1]
dataSize1 = 20
gauss1 = np.random.normal(mean1, cov1, dataSize1)
labels1 = np.full(dataSize1, 1)

mean2 = [2]
cov2 = [1]
dataSize2 = 20
gauss2 = np.random.normal(mean2, cov2, dataSize2)
labels2 = np.full(dataSize1, 2)

dataset = np.concatenate((gauss1, gauss2), axis=0)
labels = np.concatenate((labels1, labels2), axis=0)

In [3]:
# Split into test/train
data_train, data_test, labels_train, labels_test = train_test_split(dataset, labels, test_size=0.20, random_state=42)

In [4]:
class ClassDistribution:
    def __init__(self, label, mu, sigma):
        self.label = label
        self.mu = mu
        self.sigma = sigma

In [5]:
class NaiveBayesModel:

    def __init__(self):
        self.classDistributions = None
    
    def train(self, trainData, trainLabels):
        start_time = time.time()
        self.classParams = CalculateAllClassParams(trainData, trainLabels)
        return time.time() - start_time
        
    def test(self, testData, testLabels):
        if self.classParams is None:
            raise RunTimeError('You must train the model before testing')
            
        start_time = time.time()
        predictions = list(map(lambda testDataPoint: NaiveBayes(self.classParams, testDataPoint), testData))
        accuracy = accuracy_score(testLabels.tolist(), predictions)
        timing = time.time() - start_time
        return accuracy, timing

In [6]:
def CalculateAllClassParams(data, labels):
    classParams = []
    for label in set(labels):
        mu, sigma = CalculateParamsForClass([data[ii] for ii in np.where(labels == label)])
        classParams.append(ClassDistribution(label, mu, sigma))
    return classParams

In [7]:
def CalculateParamsForClass(data):
    mu = np.mean(data)
    sigma = np.cov(data)
    return mu, sigma

In [17]:
def NaiveBayes(classParams, dataPoint):
    posteriorProbabilities = {}
    for classParam in classParams:
        posteriorProbabilities[classParam.label] = norm.pdf(dataPoint, classParam.mu, classParam.sigma)
    posteriorProbabilitiesSorted = sorted(posteriorProbabilities.items(), key=operator.itemgetter(1), reverse=True)
    biggestLabelAndProbability = posteriorProbabilitiesSorted[0]
    return biggestLabelAndProbability[0]

In [18]:
model = NaiveBayesModel()
print(model.train(data_train, labels_train))
print(model.test(data_test, labels_test))

0.0
(0.625, 0.002001047134399414)
